# Mask Visualization Tool

This notebook provides an interactive tool to visualize segmentation masks and their scales from the dataset created by `make_segmentation_dataset.py`.

## Features:
- **Image Selection**: Dropdown to choose which image/camera view to display
- **Interactive Navigation**: Slider and buttons to navigate through masks sorted by scale (smallest to largest)
- **Scale Display**: Shows the current mask's scale value and position in the sequence
- **Visual Overlay**: Displays masks as semi-transparent red overlays on the original images

## Usage:
1. Load your segmentation dataset file (`.pt` file created by `make_segmentation_dataset.py`)
2. Use the dropdown to select an image
3. Use the slider or navigation buttons to browse through masks
4. Observe how the scale values change as you navigate from smallest to largest masks


In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import SfmScene and related classes
from fvdb_reality_capture.sfm_scene import SfmScene, SfmCache

# Enable widget support and test immediately
print("🔧 Setting up widget environment...")

# Test widget display immediately to verify it works
test_widget = widgets.HTML(value="<b style='color: green;'>✅ Widgets are working!</b>")
display(test_widget)

# Set matplotlib backend
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

print("📦 All imports completed successfully")
print("If you can see the green checkmark above, widgets are working properly!")


🔧 Setting up widget environment...


HTML(value="<b style='color: green;'>✅ Widgets are working!</b>")

📦 All imports completed successfully
If you can see the green checkmark above, widgets are working properly!


In [2]:
class MaskVisualizer:
    """Interactive visualizer for segmentation masks from an SfmScene.

    This class works with SfmScene objects that have been transformed with
    ComputeImageSegmentationMasksWithScales. The mask data is read from
    the SfmScene's cache.
    """

    def __init__(self, sfm_scene: SfmScene):
        """Initialize the mask visualizer with an SfmScene.

        Args:
            sfm_scene: An SfmScene that has been transformed with
                       ComputeImageSegmentationMasksWithScales.
        """
        self.sfm_scene = sfm_scene
        self.cache = sfm_scene.cache

        self.num_images = sfm_scene.num_images
        self.current_image_idx = 0
        self.current_mask_idx = 0

        # Compute the zeropad for mask file names
        self.num_zeropad = len(str(self.num_images)) + 2

        # Cache for loaded mask data to avoid repeated disk reads
        self._mask_cache: dict[int, dict] = {}

        # UI components
        self.image_selector = None
        self.mask_slider = None
        self.scale_label = None
        self.output_area = None

        print(f"✅ Loaded SfmScene with {self.num_images} images")
        print(f"📁 Cache path: {self.cache.cache_root_path}")
        print(f"📊 Files in cache: {self.cache.num_files}")

        # Check for first mask file to verify cache is correct
        first_image_id = self.sfm_scene.images[0].image_id
        test_mask_file = f"masks_{first_image_id:0{self.num_zeropad}}"
        if self.cache.has_file(test_mask_file):
            print(f"✅ Verified mask file exists: {test_mask_file}")
        else:
            print(f"⚠️ Warning: Expected mask file '{test_mask_file}' not found in cache!")
            print(f"   This might indicate the segmentation masks haven't been computed for this scene.")

    def _load_mask_data(self, image_idx: int) -> dict:
        """Load mask data for a specific image from the cache.

        Args:
            image_idx: The index of the image in the SfmScene.

        Returns:
            Dictionary with keys 'scales', 'pixel_to_mask_id', 'mask_cdf'.
        """
        if image_idx in self._mask_cache:
            return self._mask_cache[image_idx]

        # The mask files are named using image_id, not positional index
        image_id = self.sfm_scene.images[image_idx].image_id
        mask_filename = f"masks_{image_id:0{self.num_zeropad}}"
        try:
            _, data = self.cache.read_file(mask_filename)
            self._mask_cache[image_idx] = data
            return data
        except (ValueError, FileNotFoundError) as e:
            print(f"⚠️ Could not load masks for image {image_idx} (image_id={image_id}, filename={mask_filename}): {e}")
            return {}

    def _load_image(self, image_idx: int) -> np.ndarray:
        """Load the original image for a specific index.

        Args:
            image_idx: The index of the image in the SfmScene.

        Returns:
            The image as a numpy array in RGB format.
        """
        image_meta = self.sfm_scene.images[image_idx]
        image_path = image_meta.image_path

        # Load with OpenCV and convert BGR to RGB
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not load image from {image_path}")
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img_rgb

    def get_masks_for_image(self, image_idx: int):
        """Extract individual masks from pixel_to_mask_id tensor for given image.

        Args:
            image_idx: The index of the image.

        Returns:
            Tuple of (masks, mask_scales) where masks is a list of boolean tensors
            and mask_scales is a list of float scale values, sorted by scale.
        """
        data = self._load_mask_data(image_idx)
        if not data:
            return [], []

        pixel_to_mask_id = data['pixel_to_mask_id']  # [H, W, MM]
        scales = data['scales']  # [M]

        # Ensure proper dtype for indexing
        if pixel_to_mask_id.dtype in [torch.int8, torch.int16]:
            pixel_to_mask_id = pixel_to_mask_id.to(torch.int32)

        # Get unique mask IDs (excluding -1 which means no mask)
        unique_mask_ids = torch.unique(pixel_to_mask_id)
        unique_mask_ids = unique_mask_ids[unique_mask_ids >= 0]  # Remove -1

        masks = []
        mask_scales = []

        for mask_id in unique_mask_ids:
            # Create binary mask for this ID
            mask = (pixel_to_mask_id == mask_id).any(dim=-1)  # [H, W]
            masks.append(mask)

            # Get the scale for this mask ID
            if mask_id < len(scales):
                mask_scales.append(scales[mask_id].item())
            else:
                mask_scales.append(0.0)  # Fallback

        # Sort by scale (smallest to largest)
        if len(mask_scales) > 0:
            sorted_indices = np.argsort(mask_scales)
            masks = [masks[i] for i in sorted_indices]
            mask_scales = [mask_scales[i] for i in sorted_indices]

        return masks, mask_scales

    def update_display(self, change=None):
        """Update the visualization when image or mask selection changes."""
        with self.output_area:
            clear_output(wait=True)

            # Get current image and masks
            try:
                image = self._load_image(self.current_image_idx)
            except Exception as e:
                print(f"Error loading image: {e}")
                return

            masks, scales = self.get_masks_for_image(self.current_image_idx)

            # Get image metadata for display
            image_meta = self.sfm_scene.images[self.current_image_idx]
            image_name = Path(image_meta.image_path).name

            if len(masks) == 0:
                fig, ax = plt.subplots(1, 1, figsize=(12, 8))
                ax.imshow(image)
                ax.text(0.5, 0.5, 'No masks found for this image',
                       transform=ax.transAxes, ha='center', va='center', fontsize=16,
                       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
                ax.set_title(f"Image {self.current_image_idx + 1}/{self.num_images}: {image_name}")
                ax.axis('off')
                self.scale_label.value = "<b style='color: orange;'>No masks available</b>"
                plt.tight_layout()
                plt.show()
                return

            # Update mask slider range if needed
            if self.mask_slider.max != len(masks) - 1:
                self.mask_slider.max = len(masks) - 1
                self.mask_slider.value = min(self.current_mask_idx, len(masks) - 1)

            # Clamp current mask index
            self.current_mask_idx = min(self.current_mask_idx, len(masks) - 1)

            # Create visualization
            fig, ax = plt.subplots(1, 1, figsize=(12, 8))
            ax.imshow(image)

            # Overlay current mask
            if self.current_mask_idx < len(masks):
                mask = masks[self.current_mask_idx].numpy()
                scale = scales[self.current_mask_idx]

                # Create colored overlay for the mask
                masked_overlay = np.zeros((*mask.shape, 4))
                masked_overlay[mask] = [1, 0, 0, 0.4]  # Semi-transparent red
                ax.imshow(masked_overlay)

                # Update scale label
                self.scale_label.value = (
                    f"<b>Scale: {scale:.4f}</b> (Mask {self.current_mask_idx + 1}/{len(masks)}) | "
                    f"Min: {min(scales):.4f}, Max: {max(scales):.4f}"
                )

            ax.set_title(f"Image {self.current_image_idx + 1}/{self.num_images}: {image_name}")
            ax.axis('off')
            plt.tight_layout()
            plt.show()

    def on_image_change(self, change):
        """Handle image selection change."""
        self.current_image_idx = change['new']
        self.current_mask_idx = 0  # Reset to first mask
        self.update_display()

    def on_mask_change(self, change):
        """Handle mask selection change."""
        self.current_mask_idx = change['new']
        self.update_display()

    def create_widgets(self):
        """Create and display the interactive widgets."""
        # Image selector dropdown with image names
        image_options = []
        for i in range(self.num_images):
            image_name = Path(self.sfm_scene.images[i].image_path).name
            image_options.append((f"{i+1}: {image_name}", i))

        self.image_selector = widgets.Dropdown(
            options=image_options,
            value=0,
            description='Image:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        self.image_selector.observe(self.on_image_change, names='value')

        # Get initial masks to set up slider
        initial_masks, initial_scales = self.get_masks_for_image(0)
        max_masks = len(initial_masks) if initial_masks else 0

        # Mask slider
        self.mask_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=max(0, max_masks - 1),
            step=1,
            description='Mask:',
            continuous_update=True,
            layout=widgets.Layout(width='400px'),
            style={'description_width': 'initial'}
        )
        self.mask_slider.observe(self.on_mask_change, names='value')

        # Scale label
        self.scale_label = widgets.HTML(value="<b>Scale: Loading...</b>")

        # Navigation buttons
        prev_button = widgets.Button(
            description="◀ Previous",
            layout=widgets.Layout(width='100px'),
            button_style='info'
        )
        next_button = widgets.Button(
            description="Next ▶",
            layout=widgets.Layout(width='100px'),
            button_style='info'
        )

        def prev_mask(b):
            if self.mask_slider.value > 0:
                self.mask_slider.value -= 1

        def next_mask(b):
            if self.mask_slider.value < self.mask_slider.max:
                self.mask_slider.value += 1

        prev_button.on_click(prev_mask)
        next_button.on_click(next_mask)

        # Output area for plots
        self.output_area = widgets.Output()

        # Layout
        controls = widgets.VBox([
            widgets.HTML("<h3>🎛️ Mask Visualization Controls</h3>"),
            widgets.HTML(f"<p>📊 Total images: {self.num_images}</p>"),
            self.image_selector,
            widgets.HBox([prev_button, self.mask_slider, next_button]),
            self.scale_label,
            widgets.HTML("<hr>"),
            self.output_area
        ])

        display(controls)

        # Initial display
        self.update_display()

        return controls

print("✅ MaskVisualizer class defined successfully")


✅ MaskVisualizer class defined successfully


In [3]:
# ================================================================================
# CONFIGURATION - Choose ONE loading method
# ================================================================================

# OPTION 1: Load from a COLMAP dataset path
# -----------------------------------------
# If you've already run segmentation transform on a dataset, the masks are in the _cache folder.
# Set COLMAP_DATASET_PATH to your dataset directory.
COLMAP_DATASET_PATH = None  # e.g., Path("/path/to/colmap/dataset")

# OPTION 2: Load from a segmentation checkpoint
# ---------------------------------------------
# If you have a trained segmentation checkpoint, it contains the transformed SfmScene.
# Set SEGMENTATION_CHECKPOINT_PATH to your checkpoint file.
SEGMENTATION_CHECKPOINT_PATH =  Path("../nvos_results/segmentations/fern_undistort_segmentation.pt")

# Also need the GS model path when loading from checkpoint
GS_MODEL_PATH =Path("../nvos_results/reconstructions/fern_undistort.ply")

# ================================================================================
# Example configurations (uncomment one):
# ================================================================================

# Example 1: Load from a COLMAP dataset
# COLMAP_DATASET_PATH = Path("/path/to/your/colmap/dataset")

# Example 2: Load from a checkpoint
# SEGMENTATION_CHECKPOINT_PATH = Path("../garfvdb_logs/my_run/best_checkpoint.pt")
# GS_MODEL_PATH = Path("/path/to/gs_model.ply")

print("📋 Configuration:")
if COLMAP_DATASET_PATH is not None:
    print(f"  ✅ Will load from COLMAP dataset: {COLMAP_DATASET_PATH}")
elif SEGMENTATION_CHECKPOINT_PATH is not None:
    print(f"  ✅ Will load from checkpoint: {SEGMENTATION_CHECKPOINT_PATH}")
    print(f"     GS Model: {GS_MODEL_PATH}")
else:
    print("  ⚠️ No data source configured! Please set either:")
    print("     - COLMAP_DATASET_PATH for loading from a dataset")
    print("     - SEGMENTATION_CHECKPOINT_PATH + GS_MODEL_PATH for loading from a checkpoint")


📋 Configuration:
  ✅ Will load from checkpoint: ../nvos_results/segmentations/fern_undistort_segmentation.pt
     GS Model: ../nvos_results/reconstructions/fern_undistort.ply


In [ ]:
# Create and run the mask visualizer
import hashlib

def compute_gs3d_hash(gs_model) -> str:
    """Compute the hash of a GaussianSplat3d model's means tensor.

    This is the same hash used by ComputeImageSegmentationMasksWithScales
    to create cache folder names.
    """
    return hashlib.sha256(gs_model.means.detach().cpu().contiguous().numpy().tobytes()).hexdigest()


def find_mask_cache_folder(
    sfm_scene: SfmScene,
    gs3d_hash: str | None = None,
    points_per_side: int = 40,
    pred_iou_thresh: float = 0.80,
    stability_score_thresh: float = 0.80,
) -> SfmScene:
    """Find and navigate to the segmentation masks cache folder.

    When an SfmScene is loaded directly from COLMAP or restored from a checkpoint,
    its cache may be at the root level. This function navigates to the correct
    segmentation masks subfolder.

    If gs3d_hash is provided, it constructs the exact cache folder name using the
    same logic as ComputeImageSegmentationMasksWithScales. Otherwise, it searches
    for mask folders recursively.

    Args:
        sfm_scene: An SfmScene that may have masks in a cache subfolder.
        gs3d_hash: Hash of the GaussianSplat3d model's means tensor. If provided,
            used to construct the exact cache folder name.
        points_per_side: SAM2 points_per_side parameter (default: 40).
        pred_iou_thresh: SAM2 pred_iou_thresh parameter (default: 0.80).
        stability_score_thresh: SAM2 stability_score_thresh parameter (default: 0.80).

    Returns:
        A new SfmScene with the cache navigated to the masks subfolder.
    """
    cache = sfm_scene.cache

    # Check if the current cache has mask files
    # Mask files are named using image_id, check the first image's ID
    num_zeropad = len(str(sfm_scene.num_images)) + 2
    first_image_id = sfm_scene.images[0].image_id
    test_mask_file = f"masks_{first_image_id:0{num_zeropad}}"

    print(f"🔍 Checking for mask file: {test_mask_file}")
    print(f"📁 Current cache folder: {cache.cache_root_path}")
    print(f"📊 Files in current cache folder: {cache.num_files}")

    if cache.has_file(test_mask_file):
        print(f"✅ Found mask files in current cache folder")
        return sfm_scene

    cache_root = cache.cache_root_path

    # If we have the gs3d_hash, construct the exact cache folder name
    if gs3d_hash is not None:
        expected_folder_name = (
            f"segmentation_masks_scales_{gs3d_hash}"
            f"_p{points_per_side}"
            f"_i{int(pred_iou_thresh * 100)}"
            f"_s{int(stability_score_thresh * 100)}"
        )
        print(f"🔍 Looking for cache folder: {expected_folder_name}")

        # Search recursively for this exact folder
        matching_folders = list(cache_root.glob(f"**/{expected_folder_name}"))

        if matching_folders:
            mask_folder_path = matching_folders[0]
            print(f"✅ Found exact cache folder: {mask_folder_path.relative_to(cache_root)}")
        else:
            # Fall back to searching for any matching hash (different params)
            print(f"⚠️ Exact folder not found, searching for folders with matching hash...")
            hash_prefix = f"segmentation_masks_scales_{gs3d_hash}"
            matching_folders = list(cache_root.glob(f"**/{hash_prefix}*"))

            if matching_folders:
                print(f"📂 Found {len(matching_folders)} folders with matching hash:")
                for f in matching_folders:
                    print(f"    - {f.relative_to(cache_root)}")
                mask_folder_path = matching_folders[-1]
                print(f"   Using: {mask_folder_path.relative_to(cache_root)}")
            else:
                raise ValueError(
                    f"No segmentation mask cache found for gs3d_hash={gs3d_hash[:16]}...\n"
                    f"Expected folder pattern: {expected_folder_name}\n"
                    "Make sure you have run ComputeImageSegmentationMasksWithScales transform on this scene."
                )
    else:
        # No hash provided - search recursively for any mask folders
        print(f"🔍 Searching recursively for mask cache in: {cache_root}")
        mask_folders = list(cache_root.glob("**/segmentation_masks_scales_*"))

        if len(mask_folders) == 0:
            # Show what folders exist to help debugging
            all_subdirs = [d for d in cache_root.rglob("*") if d.is_dir()]
            if all_subdirs:
                print(f"📁 Found {len(all_subdirs)} subdirectories in cache:")
                for d in all_subdirs[:10]:
                    rel_path = d.relative_to(cache_root)
                    print(f"    - {rel_path}")
                if len(all_subdirs) > 10:
                    print(f"    ... and {len(all_subdirs) - 10} more")

            raise ValueError(
                f"No segmentation mask cache found in {cache_root}.\n"
                "Make sure you have run ComputeImageSegmentationMasksWithScales transform on this scene.\n"
                f"Current cache has {cache.num_files} files."
            )

        if len(mask_folders) > 1:
            print(f"⚠️ Found multiple mask cache folders (no gs3d_hash to disambiguate):")
            for folder in mask_folders:
                rel_path = folder.relative_to(cache_root)
                print(f"    - {rel_path}")
            print(f"   Using the last one: {mask_folders[-1].relative_to(cache_root)}")
        else:
            print(f"📂 Found mask cache folder: {mask_folders[0].relative_to(cache_root)}")

        mask_folder_path = mask_folders[-1]

    # Get the path from cache root to the mask folder
    relative_path = mask_folder_path.relative_to(cache_root)

    # Navigate through each folder in the path
    print(f"📂 Navigating to: {relative_path}")
    current_cache = cache
    for folder_name in relative_path.parts:
        current_cache = current_cache.make_folder(folder_name, description=f"Navigate to {folder_name}")

    # Verify the navigation worked
    print(f"📊 Files in mask cache folder: {current_cache.num_files}")

    # Create a new SfmScene with the mask cache
    return SfmScene(
        cameras=sfm_scene.cameras,
        images=sfm_scene.images,
        points=sfm_scene.points,
        points_err=sfm_scene.points_err,
        points_rgb=sfm_scene.points_rgb,
        scene_bbox=sfm_scene.scene_bbox,
        transformation_matrix=sfm_scene.transformation_matrix,
        cache=current_cache,
    )


def load_sfm_scene_from_checkpoint(checkpoint_path: Path, gs_model_path: Path) -> tuple[SfmScene, str]:
    """Load an SfmScene from a segmentation checkpoint (lightweight version).

    This function loads ONLY what's needed for mask visualization:
    - The SfmScene from the checkpoint
    - The GS model hash for cache folder lookup

    It does NOT load the full segmentation model, which saves memory.

    Args:
        checkpoint_path: Path to the segmentation checkpoint file.
        gs_model_path: Path to the GaussianSplat3d model file.

    Returns:
        Tuple of (sfm_scene, gs3d_hash) where:
            - sfm_scene: The SfmScene from the checkpoint.
            - gs3d_hash: The hash of the GaussianSplat3d model's means tensor.
    """
    from fvdb import GaussianSplat3d

    print(f"📂 Loading checkpoint from {checkpoint_path}...")
    checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)

    # Load only the GS model to compute the hash (we don't need the full segmentation model)
    print(f"📂 Loading GS model from {gs_model_path}...")
    if gs_model_path.suffix.lower() == ".ply":
        gs_model, _ = GaussianSplat3d.from_ply(gs_model_path, device=torch.device('cpu'))
    else:
        from fvdb_reality_capture.radiance_fields import GaussianSplatReconstruction
        gs_checkpoint = torch.load(gs_model_path, map_location='cpu', weights_only=False)
        runner = GaussianSplatReconstruction.from_state_dict(gs_checkpoint, device='cpu')
        gs_model = runner.model
        del runner  # Free memory

    # Compute the GS model hash (same as ComputeImageSegmentationMasksWithScales uses)
    gs3d_hash = compute_gs3d_hash(gs_model)
    print(f"🔑 Computed gs3d_hash: {gs3d_hash[:16]}...")

    # Free the GS model memory - we only needed it for the hash
    del gs_model
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # Restore just the SfmScene from the checkpoint (lightweight)
    print(f"📂 Restoring SfmScene from checkpoint...")
    if "sfm_scene" not in checkpoint:
        raise ValueError("Checkpoint does not contain 'sfm_scene' key")

    sfm_scene = SfmScene.from_state_dict(checkpoint["sfm_scene"])

    return sfm_scene, gs3d_hash


try:
    sfm_scene = None

    if COLMAP_DATASET_PATH is not None:
        # Load from COLMAP dataset
        print(f"🚀 Loading SfmScene from COLMAP dataset: {COLMAP_DATASET_PATH}")
        sfm_scene = SfmScene.from_colmap(COLMAP_DATASET_PATH)
        print(f"✅ Loaded SfmScene with {sfm_scene.num_images} images")

        # Find and navigate to the mask cache folder
        sfm_scene = find_mask_cache_folder(sfm_scene)

    elif SEGMENTATION_CHECKPOINT_PATH is not None and GS_MODEL_PATH is not None:
        # Load from checkpoint
        print(f"🚀 Loading SfmScene from checkpoint: {SEGMENTATION_CHECKPOINT_PATH}")
        sfm_scene, gs3d_hash = load_sfm_scene_from_checkpoint(
            Path(SEGMENTATION_CHECKPOINT_PATH),
            Path(GS_MODEL_PATH)
        )
        print(f"✅ Loaded SfmScene with {sfm_scene.num_images} images")

        # The checkpoint restores cache at root level, navigate to the correct mask folder
        # using the gs3d_hash to find the exact folder
        sfm_scene = find_mask_cache_folder(sfm_scene, gs3d_hash=gs3d_hash)

    else:
        raise ValueError(
            "No data source configured! Please set either:\n"
            "  - COLMAP_DATASET_PATH for loading from a dataset\n"
            "  - SEGMENTATION_CHECKPOINT_PATH + GS_MODEL_PATH for loading from a checkpoint"
        )

    # Create and display the interactive widgets
    print("🎨 Creating interactive interface...")
    visualizer = MaskVisualizer(sfm_scene)
    controls = visualizer.create_widgets()

    print("\n✅ Interactive mask visualizer loaded successfully!")
    print("Use the controls above to navigate through images and masks.")
    print("Masks are sorted from smallest to largest scale values.")

except FileNotFoundError as e:
    print(f"❌ Error: File not found - {e}")
    print("Please check that the configured paths exist.")
except ValueError as e:
    print(f"❌ Configuration error: {e}")
except Exception as e:
    import traceback
    print(f"❌ Error loading data: {e}")
    traceback.print_exc()
    print("\nMake sure:")
    print("  - The SfmScene has been transformed with ComputeImageSegmentationMasksWithScales")
    print("  - The cache folder contains the computed mask files")


🚀 Loading SfmScene from checkpoint: ../nvos_results/segmentations/fern_undistort_segmentation.pt
📂 Loading checkpoint from ../nvos_results/segmentations/fern_undistort_segmentation.pt...
📂 Loading GS model from ../nvos_results/reconstructions/fern_undistort.ply...
🔑 Computed gs3d_hash: 87d63d39daf962db...
📂 Restoring SfmScene from checkpoint...
✅ Loaded SfmScene with 20 images
🔍 Checking for mask file: masks_0000
📁 Current cache folder: /ai/segmentation_datasets/nvos/scenes/fern_undistort/_cache/cache_1
📊 Files in current cache folder: 1
🔍 Looking for cache folder: segmentation_masks_scales_87d63d39daf962dbbe227f21d897f84a33f6b439b7c765b28721011715baf033_p40_i80_s80
⚠️ Exact folder not found, searching for folders with matching hash...
📂 Found 2 folders with matching hash:
    - downsampled_2x_jpg_q95_m3/segmentation_masks_scales_87d63d39daf962dbbe227f21d897f84a33f6b439b7c765b28721011715baf033_p32_i88_s95
    - downsampled_2x_jpg_q95_m3/segmentation_masks_scales_87d63d39daf962dbbe227f2


✅ Interactive mask visualizer loaded successfully!
Use the controls above to navigate through images and masks.
Masks are sorted from smallest to largest scale values.
